# Whisper Fine-Tuning: RTX 5090 Extreme Optimization (Fixed)

**Hardware:** RTX 5090 (32GB VRAM) + 15 vCPUs
**Fixes:**
- Removed `hf_transfer` dependency.
- Added `force_download=True` to processor loading.
- Consolidated data preparation and training.

In [ ]:
# --- STEP 1: Install Dependencies ---
!pip install datasets transformers torchaudio evaluate jiwer torchcodec audiomentations tensorboard scikit-learn accelerate wandb
!pip install --upgrade transformers accelerate huggingface_hub

In [ ]:
# --- STEP 2: Imports & Configuration ---
import os
import wandb
from huggingface_hub import login

# Disable hf_transfer to prevent ModuleNotFoundError
if "HF_HUB_ENABLE_HF_TRANSFER" in os.environ:
    del os.environ["HF_HUB_ENABLE_HF_TRANSFER"]

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 🔑 Log in to Weights & Biases
wandb.login()
# os.environ["WANDB_PROJECT"] = "whisper-minang-rtx5090"

In [ ]:
# --- STEP 3: Download & Prepare Data ---
import subprocess
import tarfile
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ProcessPoolExecutor, as_completed

SAMPLING_RATE = 16000

# Working Directories
base_dir = Path("/workspace/data/audio_train/librivox-indonesia")
base_dir.mkdir(parents=True, exist_ok=True)
output_base = base_dir / "converted_wav"
output_base.mkdir(parents=True, exist_ok=True)

print("⬇️ Downloading dataset files...")
subprocess.run(["wget", "-nc", "https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/audio_train.tgz", "-P", "/workspace/data/"])
subprocess.run(["wget", "-nc", "https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/audio_test.tgz", "-P", "/workspace/data/"])
subprocess.run(["wget", "-nc", "https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/metadata_train.csv.gz", "-P", "/workspace/data/"])
subprocess.run(["wget", "-nc", "https://huggingface.co/datasets/indonesian-nlp/librivox-indonesia/resolve/main/data/metadata_test.csv.gz", "-P", "/workspace/data/"])

print("📦 Extracting audio archives...")
for archive in ["/workspace/data/audio_train.tgz", "/workspace/data/audio_test.tgz"]:
    if os.path.exists(archive):
        print(f"   Extracting {archive}...")
        with tarfile.open(archive, "r:gz") as tar:
            tar.extractall(path="/workspace/data/audio_train")

print("🎵 Starting Audio Conversion (MP3 -> WAV)...")
metadata_path = "/workspace/data/metadata_train.csv.gz"
if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Metadata file not found at {metadata_path}")

df = pd.read_csv(metadata_path)

def convert_to_wav(rel_path):
    if pd.isna(rel_path): return None
    possible_roots = [
        "/workspace/data/audio_train/librivox-indonesia",
        "/workspace/data/audio_train"
    ]
    src_path = None
    for root in possible_roots:
        temp_path = os.path.join(root, rel_path)
        if os.path.exists(temp_path):
            src_path = temp_path
            break
    if src_path is None: return None

    out_rel_path = os.path.splitext(rel_path)[0] + ".wav"
    out_path = output_base / out_rel_path
    out_path.parent.mkdir(parents=True, exist_ok=True)

    if out_path.exists(): return str(out_rel_path)

    cmd = ["ffmpeg", "-y", "-i", str(src_path), "-ac", "1", "-ar", "16000", str(out_path)]
    try:
        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
        return str(out_rel_path)
    except subprocess.CalledProcessError:
        return None

new_paths = []
with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    futures = {executor.submit(convert_to_wav, rel_path): rel_path for rel_path in df["path"].astype(str)}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Converting"):
        new_paths.append(future.result())

print("📝 Updating Metadata...")
df["path"] = new_paths
df = df.dropna(subset=["path"])
df["path"] = df["path"].apply(lambda x: os.path.join("converted_wav", str(x)) if not str(x).startswith("converted_wav") else x)

new_meta_path = base_dir / "metadata_fixed.csv"
df.to_csv(new_meta_path, index=False)
print(f"✅ Conversion Complete. Metadata saved to: {new_meta_path}")

In [ ]:
# --- STEP 4: Dataset Loading & Processor ---
import pandas as pd
import torchaudio
import torch
import numpy as np
from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split
from transformers import WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift

audio_base_path = "/workspace/data/audio_train/librivox-indonesia"
metadata_path = os.path.join(audio_base_path, "metadata_fixed.csv")
MODEL_NAME = "openai/whisper-small"
TARGET_LANGUAGE = "min"

if not os.path.exists(metadata_path):
    raise FileNotFoundError(f"❌ Metadata file not found at {metadata_path}.")

df = pd.read_csv(metadata_path)
df_lang = df[df["language"] == TARGET_LANGUAGE].copy()

if len(df_lang) == 0:
    print(f"⚠️ No samples found for '{TARGET_LANGUAGE}'. Switching to most common language.")
    most_common = df["language"].mode()[0]
    df_lang = df[df["language"] == most_common].copy()
    print(f"🔄 Switched to: '{most_common}' ({len(df_lang)} samples)")

df_lang["path"] = df_lang["path"].astype(str)
df_lang["full_path"] = df_lang["path"].apply(lambda p: os.path.join(audio_base_path, p))
df_lang = df_lang[df_lang["full_path"].apply(os.path.exists)]

print(f"✅ Final Dataset Size: {len(df_lang)} samples")

if len(df_lang) == 0:
    raise ValueError("❌ Dataset is empty! Check paths in metadata_fixed.csv vs disk.")

train_df, val_df = train_test_split(df_lang, test_size=0.1, random_state=42)
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

print("⚙️ Loading Processor (Forced Download)...")
# Use force_download to ensure fresh files and avoid OSError
try:
    feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_NAME, force_download=True)
    tokenizer = WhisperTokenizer.from_pretrained(MODEL_NAME, language="indonesian", task="transcribe", force_download=True)
    processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
except Exception as e:
    print(f"⚠️ Force download failed: {e}. Retrying with standard load...")
    processor = WhisperProcessor.from_pretrained(MODEL_NAME, language="indonesian", task="transcribe")

# Augmentation
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

def prepare_dataset_augmented(batch):
    audio, sr = torchaudio.load(batch["full_path"])
    if sr != 16000: audio = torchaudio.functional.resample(audio, sr, 16000)
    if audio.shape[0] > 1: audio = audio.mean(dim=0, keepdim=True)
    audio_np = audio.squeeze().numpy()
    augmented_audio = augment(samples=audio_np, sample_rate=16000)
    inputs = processor(audio=augmented_audio, sampling_rate=16000, text=batch["sentence"])
    return {"input_features": inputs["input_features"][0], "labels": inputs["labels"]}

def prepare_dataset_clean(batch):
    audio, sr = torchaudio.load(batch["full_path"])
    if sr != 16000: audio = torchaudio.functional.resample(audio, sr, 16000)
    if audio.shape[0] > 1: audio = audio.mean(dim=0, keepdim=True)
    inputs = processor(audio=audio.squeeze().numpy(), sampling_rate=16000, text=batch["sentence"])
    return {"input_features": inputs["input_features"][0], "labels": inputs["labels"]}

print("🔄 Processing dataset...")
train_ds = train_ds.map(prepare_dataset_augmented, num_proc=15)
val_ds = val_ds.map(prepare_dataset_clean, num_proc=15)
print("✅ Data ready.")

In [ ]:
# --- STEP 5: Metrics & Data Collator ---
import evaluate

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

@torch.no_grad()
def data_collator(features):
    input_features = [{"input_features": f["input_features"]} for f in features]
    labels = [f["labels"] for f in features]

    batch = processor.feature_extractor.pad(input_features, return_tensors="pt")
    
    # Manually pad labels
    max_len = max(len(l) for l in labels)
    padded_labels = [l + [-100]*(max_len - len(l)) for l in labels]
    batch["labels"] = torch.tensor(padded_labels, dtype=torch.long)
    
    if (batch["labels"][:, 0] == processor.tokenizer.bos_token_id).all():
        batch["labels"] = batch["labels"][:, 1:]
        
    return batch

In [ ]:
# --- STEP 6: Model Loading & Training (RTX 5090 Power) ---
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
import gc

# Cleanup memory
if 'model' in globals(): del model
if 'trainer' in globals(): del trainer
gc.collect()
torch.cuda.empty_cache()

print("🚀 Loading Model on RTX 5090...")
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME, force_download=True)
model.to("cuda")
model.config.use_cache = False

# --- ADD DROPOUT FOR REGULARIZATION ---
model.config.dropout = 0.1
model.config.attention_dropout = 0.1

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-minang-5090",
    
    # --- WANDB CONFIGURATION ---
    report_to="wandb",
    run_name="whisper-5090-run-01",
    
    # --- RTX 5090 OPTIMIZATION ---
    per_device_train_batch_size=64,   
    gradient_accumulation_steps=1,    
    gradient_checkpointing=False,     
    
    # CPU OPTIMIZATION
    dataloader_num_workers=15,        
    dataloader_pin_memory=True,       
    # -----------------------------

    learning_rate=1e-5,
    warmup_steps=20,
    max_steps=400,                    
    fp16=True,
    
    # Evaluation & Saving
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    logging_steps=10,
    predict_with_generate=True,
    generation_max_length=225,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("✅ Configured for RTX 5090.")
print(f"   Batch Size: {training_args.per_device_train_batch_size}")
print(f"   Workers:    {training_args.dataloader_num_workers}")
print("🚀 Starting Training...")
trainer.train()

In [ ]:
# --- STEP 7: Finish Run ---
wandb.finish()